In [46]:
# importing pandas as pd
from bs4 import BeautifulSoup

# Pdf checker
import re
import pdfplumber

import requests
from io import BytesIO

from urllib.request import urlopen


In [47]:
def total_values_pdf(date):
    # # Getting url link for pdf in official page
    # url = 'https://coronavirus.saude.mg.gov.br/boletim'
    # u = urlopen(url)
    # try:
    #     html = u.read().decode('utf-8')
    # finally:
    #     u.close()

    # soup = BeautifulSoup(html, "html.parser")
    # url_pdf_link = soup.find('a', string=lambda value: value and value.endswith(date))["href"]

    # Request and Open PDF file
    try:
        rq = requests.get(f"https://www.vs.saude.ms.gov.br/wp-content/uploads/2021/05/Boletim-Epidemiologico-COVID-19-2021.05.01.pdf")
        pdf_data = pdfplumber.open(BytesIO(rq.content))
    except Exception as e:
        print(e)
    
        
    # print(pdf_data.pages[0].extract_text())

    pattern_cases = r'[0-9]{3}%\s([\d]{3}\.[\d]{3})'
    pattern_deaths = r'^([\d]{1,3}\.[\d]{3})\s[\d]{1,3}\,[\d]{1,2}%'
    
    # pattern_cases = r'^([0-9]{1,3}\.[0-9]{3}\s\.[0-9]{1,3})|^([0-9]{1,3}\.[0-9]{3}\.[0-9]{1,3})|^([0-9]{1,3}\.[0-9]{1,3})'
    # pattern_deaths = r'(\b([0-9]{1,3}\.[0-9]{3}\.[0-9]{1,3})\n|\b([0-9]{1,3}\.[0-9]{1,3})\n|[0-9]{1,2}\.[0-9]\s\s[0-9]{1,3}\s\n)'
    
    # Extracting data
    text = pdf_data.pages[0].extract_text()
    total_confirmed = re.search(pattern_cases, text, re.MULTILINE)[0].split(" ")[1]
    total_death = re.search(pattern_deaths, text, re.MULTILINE)[0].split(" ")[0]
    
    # Removing dots and breaklines
    total_confirmed = re.sub(r"\.|\n|\s", "", total_confirmed)
    total_death = re.sub(r"\.|\n|\s", "", total_death)
    
    return [int(total_confirmed), int(total_death)]

In [48]:
print(total_values_pdf("01/05/2021"))

[249033, 5759]
